In [ ]:
import os
import json
from os.path import join as jp

import numpy as np
import pandas as pd
import tqdm
from skimage.morphology import binary_dilation, binary_erosion

from dpipe.io import load_json, save_json
from dpipe.medim.box import limit_box, add_margin, mask2bounding_box
from iw.cv import fill3d
from iw.xml_parsing import id2root, root2expert_roots, expert_root2nodules, nodules2centers, get_nodules
from iw.io import load_itkimage_ct, load_itkimage_lungmask, itkimage2image, load_nii, save_nii, get_ids

# 1. Unpacking the Data

In [ ]:
# from iw.path_local import luna_raw_path
from iw.path import luna_raw_path
raw_dp = luna_raw_path

xml_dp = jp(raw_dp, 'tcia-lidc-xml/')
lung_mask_dp = jp(raw_dp, 'seg-lungs-LUNA16/')
nodules_dp = jp(raw_dp, 'annotations.csv')

In [ ]:
nodules_df = pd.read_csv(nodules_dp, index_col='seriesuid')
nodules_df.head()

In [ ]:
n_subsets = 10
n_train, n_val, n_holdout = 6, 2, 2

subset_ids = np.arange(n_subsets)

ids = dict()
ids['train'], ids['val'], ids['holdout'] = map(lambda x: get_ids(raw_dp, x), (subset_ids[:n_train], 
                                                                              subset_ids[n_train:-n_holdout], 
                                                                              subset_ids[-n_holdout:]))
with open('reverse_ids.json', 'r') as f:
    reverse_ids = json.load(f)

In [ ]:
def get_absolute_centers(_id: str, nodules_df: pd.DataFrame) -> (list, bool):
    """Gets absolute coordinates of nodule centers by id."""

    abs_coords = []
    no_nodules = True

    if _id in nodules_df.index:
        abs_coords = nodules_df[['coordX', 'coordY', 'coordZ']].loc[_id].values
        no_nodules = False

        if len(np.shape(abs_coords)) == 1:
            abs_coords = np.array([abs_coords])

    return abs_coords, no_nodules


def absolute2relative(abs_coords: np.ndarray, origin: np.ndarray, spacing: np.ndarray) -> list:
    """Shifts absolute coordinates into the relative."""

    rel_coords = []

    for abs_c in abs_coords:
        rel_c = np.int64(np.round((abs_c - origin) / spacing))
        assert np.all(rel_c > 0)
        rel_coords.append(rel_c)

    return rel_coords


def center2hit(center: list, rel_centers: list, r: int) -> bool:
    is_hit = False
    for rel_center in rel_centers:
        if np.linalg.norm(np.array(center) - np.array(rel_center)) <= r:
            is_hit = True

    return is_hit


def nodules2target(target: np.ndarray, expert_nodules: list, rel_centers: list,
                   origin: np.ndarray, spacing: np.ndarray, r=10) -> np.ndarray:
    """Builds segmentation mask from the given expert delineation."""

    for nodules in expert_nodules:
        centers = nodules2centers(nodules, origin[-1], spacing[-1])

        for center, nodule in zip(centers, nodules):
            if center2hit(center, rel_centers, r=r):
                target += fill3d(np.zeros_like(target), nodule, origin[-1], spacing[-1])
                
    n_experts = 4

    return np.float32(target / n_experts >= 0.5)

In [ ]:
def get_set(_id: str, raw_dp: str, xml_dp: str, lung_mask_dp: str):
    
    # CT
    itkimage_ct = load_itkimage_ct(_id, raw_dp)
    ct = itkimage2image(itkimage_ct)
    
    
    # lung_mask
    itkimage_lung_mask = load_itkimage_lungmask(_id, lung_mask_dp)
    lung_mask = itkimage2image(itkimage_lung_mask)
    
    
    # origin, spacing
    origin, spacing = map(np.array, (itkimage_ct.GetOrigin(), itkimage_ct.GetSpacing()))

    
    # "tumor_data": nodules' centers, diameters
    abs_centers, no_tumor = get_absolute_centers(_id, nodules_df)
    if _id in reverse_ids:
        abs_centers[:, :2] = 2 * origin[:2] - abs_centers[:, :2]
        
    rel_centers = absolute2relative(abs_centers, origin, spacing)
    
    if not no_tumor:
        try:
            diameters = list(nodules_df.loc[_id]['diameter_mm'])
        except TypeError:
            diameters = list([nodules_df.loc[_id]['diameter_mm']])
    else:
        diameters = []
        
    tumor_data = [{'center': abs_c, 'diameter': d} for abs_c, d in zip(abs_centers, diameters)]
    
    
    # target    
    target = np.zeros_like(ct, dtype='float32')
    if not no_tumor:
        expert_nodules = get_nodules(_id, xml_dp)
        target = nodules2target(target, expert_nodules, rel_centers, origin, spacing)
    
    
    return ct, target, lung_mask, origin, spacing, tumor_data

# 2. Preprocessing steps

In [ ]:
def drop_trachea(lung_mask: np.ndarray) -> np.ndarray:
    """Excludes trachea's class from the lungs\' mask."""

    labels, volumes = np.unique(lung_mask, return_counts=True)
    trachea_label = labels[np.argmin(volumes)]
    lung_mask[lung_mask == trachea_label] = 0

    return lung_mask


def mask2bbox(mask: np.ndarray, margin=0) -> np.ndarray:
    """Creates valid bounding box with a specified margin."""

    box = limit_box(add_margin(mask2bounding_box(mask), margin), mask.shape)
    return box


def crop2box(image: np.ndarray, box: np.ndarray) -> np.ndarray:
    return image[box[0, 0]:box[1, 0],
                 box[0, 1]:box[1, 1],
                 box[0, 2]:box[1, 2]]


def mask2closed_mask(mask: np.ndarray, margin=1) -> np.ndarray:
    """Creates closed mask around the given mask with specified margin."""

    if margin > 0:
        for _ in range(margin):
            mask = binary_dilation(mask)

        for _ in range(margin):
            mask = binary_erosion(mask)

    return mask.astype('int64')


def mask2dilated_mask(mask: np.ndarray, margin=1) -> np.ndarray:
    """Creates closed mask around the given mask with specified margin."""

    if margin > 0:
        for _ in range(margin):
            mask = binary_dilation(mask)

    return mask.astype('int64')

In [ ]:
def preprocess_set(_id, ct, target, lung_mask, origin, spacing, tumor_data):    
    lung_mask = mask2dilated_mask(mask2closed_mask(drop_trachea(lung_mask), margin=10), margin=3)
    
    
    box = mask2bbox(lung_mask, margin=5)
    cropped_ct, cropped_target, cropped_lung_mask = [crop2box(scan, box) for scan in [ct, target, lung_mask]]
    
    
    shifted_tumor_data = []
    if np.any(tumor_data):
        for tumor in tumor_data:
            abs_c, d = tumor['center'], tumor['diameter'] 
            shifted_tumor_data.append({'center': list(abs_c - box[0] * spacing + origin), 
                                       'diameter': d,
                                       'abs_shift_mm': box[0] * spacing - origin})
            
    return cropped_ct, cropped_target, cropped_lung_mask, origin, spacing, shifted_tumor_data

# 3. Dump the Data

In [ ]:
def create_record(split: str, save_dp: str, _id: str, 
                  ct: np.ndarray, target: np.ndarray, lung_mask: np.ndarray, 
                  origin: np.ndarray, spacing: np.ndarray, tumor_data: list) -> dict:
    os.makedirs(jp(save_dp, split, _id), exist_ok=True)

    record = {
        'id': _id,
        'CT': jp(_id, 'CT.nii.gz'),
        'target': jp(_id, 'target.nii.gz'),
        'lung_mask': jp(_id, 'lung_mask.nii.gz'),
        'origin': jp(_id, 'origin.json'),
        'spacing': jp(_id, 'spacing.json'),
        'n_tumors': len(tumor_data) if np.any(tumor_data) else 0,
        'tumor_data': jp(_id, 'tumors.json'),
        'split': split
    }

    save_nii(jp(save_dp, split, record['CT']), np.array(ct, dtype='float32'))
    save_nii(jp(save_dp, split, record['target']), np.array(target, dtype='float32'))
    save_nii(jp(save_dp, split, record['lung_mask']), np.array(lung_mask, dtype='float32'))

    save_json(origin, jp(save_dp, split, record['origin']))
    save_json(spacing, jp(save_dp, split, record['spacing']))
    save_json(tumor_data, jp(save_dp, split, record['tumor_data']))

    return record


def dump(split: str, save_dp: str, ids=ids, raw_dp=raw_dp, xml_dp=xml_dp, lung_mask_dp=lung_mask_dp,  
         preprocessing=True) -> None:
    
    records = []
    
    with open('outlier_ids.json', 'r') as f:
        outlier_ids = json.load(f)
    
    for _id in tqdm.notebook.tqdm(ids[split]):
        if _id not in outlier_ids:
            _set = get_set(_id, raw_dp, xml_dp, lung_mask_dp)
            if preprocessing:
                _set = preprocess_set(_id, *_set)

            record = create_record(split, save_dp, _id, *_set)
            records.append(record)

    metadata = pd.DataFrame.from_records(records, index='id')
    metadata.to_csv(path_or_buf=jp(save_dp, split, 'metadata.csv'))

In [ ]:
# from iw.path_local import luna_data_path
from iw.path import luna_data_path
save_dp = luna_data_path

dump('train', save_dp, preprocessing=True)
dump('val', save_dp, preprocessing=True)
dump('holdout', save_dp, preprocessing=True)

# 4. Merge metadata

In [ ]:
df_t = pd.read_csv(jp(save_dp, 'train/metadata.csv'), index_col='id')
df_v = pd.read_csv(jp(save_dp, 'val/metadata.csv'), index_col='id')
df_h = pd.read_csv(jp(save_dp, 'holdout/metadata.csv'), index_col='id')
df = pd.concat((df_t, df_v, df_h))

for _id in df.index:
    row = df.loc[_id]
    for c in df.columns:
        if c != 'n_tumors' and c != 'split':
            df[c].loc[_id] = row['split'] + '/' + row[c]
            
df.to_csv(jp(save_dp, 'metadata.csv'), index_label='id')